In [1]:
from taxcalc import *
import pandas as pd
import numpy as np
import os

In [2]:
pol = Policy()
# To make sure that Tax-Calculator uses the PUF,
# add puf.csv to this folder and initiate the Records
# class with recs=Records()
recs = Records()
calc = Calculator(pol, recs)

In [3]:
# Use Tax-Calculator to extrapolate data to 2017
calc.advance_to_year(2017)
calc.calc_all()
# By toggling the all_vars argument, we tell Tax-Calculator
# to construct a dataframe using all PUF variables and all
# calculated variables
puf_cps_2017 = calc.dataframe(variable_list=[], all_vars=True)
len(puf_cps_2017.index)

248591

In [4]:
# filter out filers imputed from CPS
puf_2017 = puf_cps_2017.copy()
puf_2017 = puf_2017.loc[puf_2017['data_source'] == 1]
len(puf_2017.index)

241245

In [5]:
# Check that number of PUF vars + number of calculated
# vars = number of columns in our dataframe
puf_vars = recs.USABLE_READ_VARS
calc_vars = recs.CALCULATED_VARS
print("Num PUF vars: {}".format(len(puf_vars)))
print("Num calculated vars: {}".format(len(calc_vars)))
print("Num columns in puf_2017: {}".format(len(puf_2017.columns)))

Num PUF vars: 106
Num calculated vars: 100
Num columns in puf_2017: 206


In [6]:
target_vars_pos = ['c00100']
target_vars = ['e00200', 'c04470', 'c17000', 'c04800', 'c05800', 'c09600', 'e00700', 'c01000']

In [7]:
# add indicator for households where c00100 > 0 (to line up which HT2)
for var in target_vars_pos:
    count_var = var + '_n'
    puf_2017[count_var] = np.where(puf_2017[var] > 1, 1, 0)

# add indicator for households where other target vars != 0
for var in target_vars:
    count_var = var + '_n'
    puf_2017[count_var] = np.where(puf_2017[var] != 0., 1, 0)

# add indicator for filing status
puf_2017['mars_1_n'] = np.where(puf_2017.MARS == 1, 1, 0)
puf_2017['mars_2_n'] = np.where(puf_2017.MARS == 2, 1, 0)

In [8]:
HT2_AGI_STUBS = [-9e99, 1.0, 10e3, 25e3, 50e3,
                75e3, 100e3, 200e3, 500e3, 1e6, 9e99]

# sort PUF filers into AGI bins (same bins as HT2)
puf_2017['agi_stub'] = pd.cut(puf_2017['c00100'], HT2_AGI_STUBS, labels=list(range(1,11)), right=False)

In [9]:
# NOTE: before reading csv, I cleared number formatting in excel to get rid of thousands comma separator
ht2 = pd.read_csv('17in54cmcsv.csv')

# add column in ht2 for filers with positive AGI
ht2['pos_AGI']  = np.where(ht2['AGI_STUB'] == 0, \
                              int(ht2.loc[0, 'N1']) - \
                              int(ht2.loc[1, 'N1']), \
                              np.where(ht2['AGI_STUB'] == 1, \
                                       0, ht2['N1']))

# filter for US
ht2_us = ht2[ht2['STATE']=='US']
# ht2[ht2['AGI_STUB']==1]

In [10]:
crosswalk = [('mars_1_n', 'MARS1'), # Single returns number
           ('mars_2_n', 'MARS2'), # Joint returns number          
           ('c00100', 'A00100'), # AGI amount
           ('c00100_n', 'pos_AGI'), # AGI amount  
           ('e00200', 'A00200'), # Salary and wage amount            
           ('e00200_n', 'N00200'), # Salary and wage number           
           ('c01000', 'A01000'), # Capital gains amount             
           ('c01000_n', 'N01000'), # Capital gains number         
           ('c04470', 'A04470'), # Itemized deduction amount (0 if standard deduction)      
           ('c04470_n', 'N04470'), # Itemized deduction number (0 if standard deduction)
           ('c17000', 'A17000'), # Medical expenses deducted amount
           ('c17000_n', 'N17000'), # Medical expenses deducted number
           ('c04800', 'A04800'), # Taxable income amount
           ('c04800_n', 'N04800'), # Taxable income number
           ('c05800', 'A05800'), # Regular tax before credits amount
           ('c05800_n', 'N05800'), # Regular tax before credits amount
           ('c09600', 'A09600'), # AMT amount
           ('c09600_n', 'N09600'), # AMT number
           ('e00700', 'A00700'), # SALT amount
           ('e00700_n', 'N00700') # SALT number
]

In [11]:
# create table to compare PUF sums with HT2 vals
compare_df = pd.DataFrame()

for item in crosswalk:
    if item[1].startswith('A'):
        # HT2 amounts are in thousands
        ht2_val = round(int(ht2_us.loc[0, item[1]]) * 1000, 1)        
    else:
        ht2_val = round(int(ht2_us.loc[0, item[1]]), 1)
        
    puf_var = item[0]
    puf_val = (puf_2017[puf_var] * puf_2017['s006']).sum()
    perc_dif = round(((puf_val - ht2_val) / ht2_val), 3)
    data = pd.Series(dtype='float64')
    data = pd.Series(data = {"HT2_value": ht2_val / 1e6, \
                             "PUF_val": puf_val / 1e6, \
                             "perc_dif": perc_dif}, name=item[0])
    
    compare_df = compare_df.append(data, ignore_index=False)
    
compare_df

,HT2_value,PUF_val,perc_dif
mars_1_n,7.285867e+01,7.280079e+01,-0.001
mars_2_n,5.471960e+01,5.525759e+01,0.010
c00100,1.099139e+07,1.127737e+07,0.026
c00100_n,1.503518e+02,1.509270e+02,0.004
e00200,7.557396e+06,7.677457e+06,0.016
e00200_n,1.258107e+02,1.211315e+02,-0.037
c01000,8.557280e+05,8.443012e+05,-0.013
c01000_n,2.549433e+01,1.956296e+01,-0.233
c04470,1.403313e+06,1.306515e+06,-0.069
c04470_n,4.710365e+01,4.622955e+01,-0.019


In [12]:
keep_list = ['agi_stub', 's006']
var_list = []
for item in crosswalk:
    keep_list.append(item[0])
    var_list.append(item[0])

In [13]:
# calculate PUF target var totals by agi_stub
puf_summary_temp = puf_2017.copy()
puf_summary_temp = puf_summary_temp[keep_list]

for var in var_list:
    if var.endswith('_n'):
        new_var = var + 'w'
        # when calculating totals, multiply by weight
        puf_summary_temp[new_var] = puf_summary_temp[var] * puf_summary_temp['s006']
    else:
        new_var = var + '_sum'
        puf_summary_temp[new_var] = puf_summary_temp[var] * puf_summary_temp['s006']
        
# drop unweighted totals
puf_summary_temp.drop(var_list, axis=1, inplace=True)
puf_summary = puf_summary_temp.groupby(['agi_stub']).sum()

puf_summary

,s006,mars_1_nw,mars_2_nw,c00100_sum,c00100_nw,e00200_sum,e00200_nw,c01000_sum,c01000_nw,c04470_sum,...,c17000_sum,c17000_nw,c04800_sum,c04800_nw,c05800_sum,c05800_nw,c09600_sum,c09600_nw,e00700_sum,e00700_nw
agi_stub,,,,,,,,,,,,,,,,,,,,,
1,2.654133e+06,1.855077e+06,574881.18,-8.360921e+10,0.000000e+00,1.287505e+10,4.228054e+05,5.817276e+09,386651.29,5.392992e+07,...,6.500608e+06,1277.93,0.000000e+00,0.000000e+00,4.521082e+09,1.249949e+04,4.521082e+09,12499.49,3.892221e+08,84915.27
2,2.375883e+07,1.937943e+07,2050731.21,1.143506e+11,2.375883e+07,7.955246e+10,1.618520e+07,1.000895e+09,1305162.20,7.914193e+08,...,6.299525e+07,21521.08,3.896924e+09,2.281909e+06,6.188404e+08,1.888343e+06,3.288150e+08,8141.76,2.105260e+08,271326.11
3,3.095294e+07,1.764747e+07,4784323.47,5.247338e+11,3.095294e+07,3.578740e+11,2.261752e+07,4.190977e+09,1524695.71,2.270455e+10,...,7.859685e+09,897892.31,1.122300e+11,1.818672e+07,1.188120e+10,1.761476e+07,6.056414e+08,18531.62,3.772692e+08,595870.32
4,3.312858e+07,1.664223e+07,8353844.26,1.201652e+12,3.312858e+07,9.356204e+11,2.768108e+07,1.264258e+10,2264397.80,9.780523e+10,...,2.128523e+10,2459996.59,6.116926e+11,3.182400e+07,7.363903e+10,3.146752e+07,4.949938e+08,16918.43,1.648897e+09,2366916.70
5,2.076448e+07,9.064928e+06,8292795.24,1.277637e+12,2.076448e+07,9.514498e+11,1.730679e+07,1.688496e+10,2413872.06,1.428515e+11,...,2.095861e+10,2296516.16,8.394107e+11,2.064840e+07,1.174900e+11,2.056768e+07,6.359772e+08,19971.17,3.122033e+09,3734495.41
6,1.298458e+07,3.766553e+06,7829699.26,1.129633e+12,1.298458e+07,8.148434e+11,1.085177e+07,1.765007e+10,2237093.56,1.498831e+11,...,1.615442e+10,1703199.76,7.936286e+11,1.294417e+07,1.193223e+11,1.290033e+07,7.797482e+08,78359.07,4.074321e+09,3813797.39
7,2.105216e+07,3.477483e+06,16355086.01,2.867115e+12,2.105216e+07,2.202470e+12,1.869731e+07,5.475296e+10,4975253.50,4.227093e+11,...,2.184877e+10,1949401.98,2.159879e+12,2.102729e+07,3.769448e+11,2.099546e+07,3.138220e+09,706446.62,1.182075e+10,8970211.79
8,6.663056e+06,7.706790e+05,5647475.90,1.892157e+12,6.663056e+06,1.350277e+12,6.005053e+06,1.000587e+11,3196159.30,2.667955e+11,...,6.699319e+09,302857.81,1.555506e+12,6.659706e+06,3.691453e+11,6.660554e+06,2.174622e+10,3887446.69,6.213846e+09,2696603.65
9,1.080542e+06,1.283942e+05,918495.30,7.329336e+11,1.080542e+06,4.262318e+11,9.256024e+05,8.690843e+10,784619.77,7.760433e+10,...,7.845222e+08,14267.22,6.547849e+11,1.079632e+06,1.916856e+11,1.080461e+06,7.412631e+09,505856.92,1.602055e+09,274674.35


In [14]:
# print out national totals of target vars

for column in puf_summary.columns:
    targ_total = round(puf_summary[column].sum() / 1e6, 1)
#     print("{} (mil): {}".format(column, targ_total))

In [15]:
ht2_keep_list = ['STATE', 'AGI_STUB']
for item in crosswalk:
    ht2_keep_list.append(item[1])

In [23]:
# calculate ratios of state totals to national totals by agi_stub for each state

states = list(ht2.STATE.unique())
# remove US from list of states
states.pop(0)

ht2_us_vals = ht2_us.drop(['STATE', 'AGI_STUB'], axis=1)
# create empty df for ratios
ratio_df = pd.DataFrame()

for state in states:
    state_df = ht2[ht2['STATE']==state].reset_index()
    state_id = state_df[['STATE', 'AGI_STUB']]
    state_vals = state_df.drop(['index', 'STATE', 'AGI_STUB'], axis=1)
    
    # divide each state's total/stub by the U.S. total/stub
    ratios = state_vals / ht2_us_vals
    # tack back on states and stubs
    ratios_state = pd.concat([state_id, ratios], axis=1)
    # add each state ratio df to overall ratio df
    ratio_df = pd.concat([ratio_df, ratios_state])
    
ratio_df = ratio_df[ht2_keep_list]
ratio_df

,STATE,AGI_STUB,MARS1,MARS2,A00100,pos_AGI,A00200,N00200,A01000,N01000,...,A17000,N17000,A04800,N04800,A05800,N05800,A09600,N09600,A00700,N00700
0,AL,0,0.011741,0.013855,0.010815,1.000000,0.011116,0.013578,0.006543,0.009330,...,0.013343,0.016541,0.010135,0.012872,0.009141,0.012927,0.003393,0.004806,0.010377,0.015042
1,AL,1,0.011053,0.014484,0.007382,NaN,0.008540,0.009334,0.005168,0.010007,...,NaN,NaN,NaN,NaN,0.009211,0.024748,0.008319,0.009124,0.008338,0.013401
2,AL,2,0.012756,0.014862,0.014341,0.013567,0.013852,0.014204,0.010218,0.009625,...,0.014713,0.015183,0.012510,0.012589,0.015782,0.013418,0.165184,0.007937,0.011848,0.014667
3,AL,3,0.013526,0.016522,0.016329,0.016478,0.016867,0.016838,0.009238,0.009690,...,0.017206,0.019656,0.013472,0.014365,0.013594,0.014517,0.019371,0.010221,0.014165,0.017328
4,AL,4,0.011531,0.015854,0.013727,0.013828,0.013693,0.013704,0.009254,0.009841,...,0.016507,0.020044,0.013057,0.013851,0.012887,0.013873,0.009408,0.006536,0.016563,0.018488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,OA,6,0.003212,0.003087,0.003512,0.003519,0.004287,0.003427,0.009384,0.005038,...,0.001118,0.000866,0.003702,0.003520,0.004159,0.003510,0.006442,0.011536,0.001187,0.000746
7,OA,7,0.004783,0.002616,0.003583,0.003537,0.004272,0.003467,0.007690,0.004515,...,0.001261,0.000939,0.003901,0.003539,0.004588,0.003541,0.003830,0.006261,0.001337,0.000896
8,OA,8,0.009058,0.003893,0.005736,0.005465,0.007136,0.005391,0.007596,0.005430,...,0.001622,0.001619,0.006353,0.005468,0.007114,0.005470,0.001920,0.002917,0.002587,0.001750
9,OA,9,0.013611,0.007231,0.009874,0.009676,0.013618,0.009877,0.008555,0.008901,...,0.002610,0.002538,0.010774,0.009682,0.011585,0.009674,0.002545,0.005044,0.004090,0.003535


In [17]:
# verify that ratios add up to 1 by AGI_STUB

# ratio_df.groupby(['AGI_STUB']).sum()

In [18]:
# rename the PUF variables so they line up with HT2

rename_dict = {'mars_1_nw': 'MARS1', # Single returns number
           'mars_2_nw': 'MARS2', # Joint returns number          
           'c00100_sum': 'A00100', # AGI amount
           'c00100_nw': 'pos_AGI', # AGI amount  
           'e00200_sum': 'A00200', # Salary and wage amount            
           'e00200_nw': 'N00200', # Salary and wage number           
           'c01000_sum': 'A01000', # Capital gains amount             
           'c01000_nw': 'N01000', # Capital gains number         
           'c04470_sum': 'A04470', # Itemized deduction amount (0 if standard deduction)      
           'c04470_nw': 'N04470', # Itemized deduction number (0 if standard deduction)
           'c17000_sum': 'A17000', # Medical expenses deducted amount
           'c17000_nw': 'N17000', # Medical expenses deducted number
           'c04800_sum': 'A04800', # Taxable income amount
           'c04800_nw': 'N04800', # Taxable income number
           'c05800_sum': 'A05800', # Regular tax before credits amount
           'c05800_nw': 'N05800', # Regular tax before credits amount
           'c09600_sum': 'A09600', # AMT amount
           'c09600_nw': 'N09600', # AMT number
           'e00700_sum': 'A00700', # SALT amount
           'e00700_nw': 'N00700'} # SALT number

In [19]:
# multiply state/stub ratios from HT2 by PUF stub totals
puf_sum_vals = puf_summary.drop(['s006'], axis=1)
puf_sum_vals = puf_sum_vals.rename(columns=rename_dict)
ratio_vals_df = pd.DataFrame()

for stub in range(1,11):
    # filter ratio df by AGI stub
    stub_ratio = ratio_df[ratio_df['AGI_STUB']==stub]
    stub_vals = stub_ratio.drop(['STATE', 'AGI_STUB'], axis=1)
    # multiply the ratios for each stub by the PUF totals
    ratio_vals_bin = puf_sum_vals.loc[stub] * stub_vals
    ratio_vals_bin = pd.concat([stub_ratio.STATE, ratio_vals_bin], axis=1)
    ratio_vals_df = pd.concat([ratio_vals_df, ratio_vals_bin])
    
ratio_vals_df = ratio_vals_df.reset_index()
ratio_vals_df = ratio_vals_df.rename(columns={'index': 'AGI_STUB'})
ratio_vals_df

,AGI_STUB,STATE,MARS1,MARS2,A00100,pos_AGI,A00200,N00200,A01000,N01000,...,A17000,N17000,A04800,N04800,A05800,N05800,A09600,N09600,A00700,N00700
0,1,AL,20503.627038,8326.693444,-6.171778e+08,NaN,1.099590e+08,3946.426498,3.006089e+07,3869.026848,...,NaN,NaN,NaN,NaN,4.164146e+07,309.336599,3.761041e+07,114.046442,3.245512e+06,1137.917819
1,1,AK,3049.030745,939.689542,-1.052300e+08,NaN,2.573906e+07,772.620927,4.040524e+06,625.421139,...,NaN,NaN,NaN,NaN,4.642667e+06,60.148783,4.507123e+06,22.809288,3.952433e+04,29.556307
2,1,AZ,39210.240788,13468.883440,-1.882839e+09,NaN,2.027141e+08,7233.852793,1.152927e+08,9491.685515,...,NaN,NaN,NaN,NaN,4.876836e+07,186.174805,4.765298e+07,205.283595,7.305226e+06,2630.511323
3,1,AR,12475.985705,6751.843378,-5.361718e+08,NaN,7.910751e+07,3060.184846,2.279087e+07,2924.763560,...,NaN,NaN,NaN,NaN,1.138268e+07,85.926833,9.976932e+06,68.427865,1.733424e+06,783.242135
4,1,CA,254380.487755,67448.827149,-1.317660e+10,NaN,1.454452e+09,48690.267805,9.206684e+08,54902.165443,...,NaN,NaN,NaN,NaN,7.703365e+08,1268.852903,7.886590e+08,1984.408084,8.960392e+07,15930.849471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,10,WA,1757.473874,11488.880085,4.009686e+10,13693.942517,1.180281e+10,11109.146609,1.625319e+10,12039.569535,...,5.936310e+06,61.003505,3.891933e+10,13685.198912,1.143775e+10,13692.946641,1.264018e+08,2161.023969,3.242073e+07,1360.301333
516,10,WV,93.111199,631.616156,1.404956e+09,754.698110,3.811830e+08,614.778016,2.918220e+08,639.733224,...,0.000000e+00,0.000000,1.329992e+09,755.422980,4.302642e+08,755.246457,6.318087e+06,114.239153,6.563643e+06,329.513988
517,10,WI,698.333990,5477.636666,1.581317e+10,6321.963878,4.732641e+09,5446.717512,4.187402e+09,5558.337852,...,3.336553e+06,24.401402,1.490945e+10,6317.087818,4.770084e+09,6326.557281,6.056017e+07,1085.271949,7.934852e+07,3016.320348
518,10,WY,151.305698,773.185295,4.826902e+09,973.451185,6.314641e+08,647.134754,2.514670e+09,880.944112,...,0.000000e+00,0.000000,4.506049e+09,963.438003,1.227933e+09,974.158474,2.472490e+07,247.518164,1.075188e+07,202.777838


In [20]:
# look at people in the PUF with negative AGI and positive itemized deductions
# puf_2017[(puf_2017['c00100']<1) & (puf_2017['c04470']>0)]

In [21]:
# sanity check -- compare sums by AGI stub
round(ratio_vals_df.groupby(['AGI_STUB']).sum() - puf_sum_vals, 0)

,MARS1,MARS2,A00100,pos_AGI,A00200,N00200,A01000,N01000,A04470,N04470,A17000,N17000,A04800,N04800,A05800,N05800,A09600,N09600,A00700,N00700
AGI_STUB,,,,,,,,,,,,,,,,,,,,
1,0.0,52.0,0.0,0.0,0.0,23.0,0.0,18.0,-53929915.0,-12178.0,-6500608.0,-1278.0,0.0,0.0,-0.0,11.0,0.0,0.0,0.0,15.0
2,0.0,13.0,0.0,-12.0,0.0,-11.0,0.0,0.0,-0.0,6.0,0.0,1.0,0.0,-20.0,0.0,41.0,0.0,-129.0,0.0,19.0
3,0.0,31.0,0.0,29.0,0.0,27.0,0.0,7.0,0.0,14.0,-0.0,19.0,0.0,27.0,-0.0,26.0,0.0,126.0,0.0,-0.0
4,9.0,48.0,0.0,37.0,-0.0,44.0,0.0,7.0,0.0,26.0,0.0,29.0,0.0,46.0,0.0,18.0,0.0,88.0,-0.0,22.0
5,62.0,9.0,0.0,30.0,-0.0,10.0,0.0,-7.0,0.0,0.0,-0.0,22.0,0.0,40.0,0.0,20.0,-0.0,7.0,-0.0,45.0
6,40.0,19.0,0.0,58.0,0.0,-19.0,0.0,22.0,0.0,10.0,0.0,-0.0,-0.0,19.0,0.0,0.0,0.0,19.0,0.0,34.0
7,43.0,21.0,-0.0,42.0,-0.0,32.0,0.0,30.0,0.0,11.0,0.0,11.0,0.0,74.0,0.0,-21.0,-0.0,51.0,-0.0,48.0
8,-11.0,43.0,0.0,43.0,0.0,87.0,0.0,51.0,0.0,0.0,-0.0,48.0,0.0,-0.0,-0.0,0.0,0.0,32.0,0.0,0.0
9,23.0,-11.0,0.0,-22.0,-0.0,22.0,0.0,19.0,0.0,22.0,-0.0,36.0,0.0,54.0,0.0,65.0,-0.0,20.0,-0.0,24.0


In [22]:
# sanity check -- compare sums by state
ht2_vals_check = ht2[ht2_keep_list][ht2['AGI_STUB'] != 0]
df1 = ratio_vals_df.groupby(['STATE']).sum()
df2 = ht2_vals_check.groupby(['STATE']).sum()
# (df1 - df2)/df1